# normalize_dfs_xgb_classifier

In [1]:
from greenguard.demo import load_demo

target_times, readings = load_demo()

In [2]:
pipeline_name = 'classes.normalize_dfs_xgb_classifier'

In [3]:
from greenguard.pipeline import GreenGuardPipeline

pipeline = GreenGuardPipeline(pipeline_name)

In [4]:
pipeline.template['primitives']

['pandas.DataFrame.resample',
 'featuretools.EntitySet.entity_from_dataframe',
 'featuretools.EntitySet.normalize_entity',
 'featuretools.EntitySet.normalize_entity',
 'featuretools.dfs',
 'mlprimitives.custom.feature_extraction.CategoricalEncoder',
 'xgboost.XGBClassifier']

# Step by Step execution

## Input Data

In [5]:
readings.head()

,turbine_id,signal_id,timestamp,value
0,T001,S01,2013-01-10,323.0
1,T001,S02,2013-01-10,320.0
2,T001,S03,2013-01-10,284.0
3,T001,S04,2013-01-10,348.0
4,T001,S05,2013-01-10,273.0


In [6]:
target_times.head()

,turbine_id,cutoff_time,target
0,T001,2013-01-12,0
1,T001,2013-01-13,0
2,T001,2013-01-14,0
3,T001,2013-01-15,1
4,T001,2013-01-16,0


## Data Preparation (part of GreenGuard Pipeline)

* Input: target_times, readings, turbines
* Output: X, y, readings, turbines
* Effect: target_times has been split into X and y

## pandas.DataFrame.resample

* Input: readings
* Output: readings (resampled)
* Effect: readings have been resampled to the indicated resample rule and turbine_id,
  signal_id and timestamp have been set as a multi-index

In [7]:
step = 0
context = pipeline.fit(target_times, readings, output_=step)

In [8]:
context.keys()

dict_keys(['readings', 'turbines', 'X', 'y'])

In [9]:
context['readings'].head()

,turbine_id,signal_id,timestamp,value
0,T001,S01,2013-01-10 00:00:00,323.0
1,T001,S01,2013-01-10 00:10:00,346.0
2,T001,S01,2013-01-10 00:20:00,407.0
3,T001,S01,2013-01-10 00:30:00,257.0
4,T001,S01,2013-01-10 00:40:00,267.0


##  featuretools.EntitySet.entity_from_dataframe

* Input: readings (resampled)
* Output: entityset
* Effect: Entityset has been generated from readings

In [10]:
step = 1
context = pipeline.fit(**context, output_=step, start_=step)

In [11]:
context.keys()

dict_keys(['readings', 'turbines', 'X', 'y', 'entityset'])

In [12]:
context['entityset']

Entityset: entityset
  Entities:
    readings [Rows: 1329146, Columns: 5]
  Relationships:
    No relationships

In [13]:
context['readings'].head()

,turbine_id,signal_id,timestamp,value
0,T001,S01,2013-01-10 00:00:00,323.0
1,T001,S01,2013-01-10 00:10:00,346.0
2,T001,S01,2013-01-10 00:20:00,407.0
3,T001,S01,2013-01-10 00:30:00,257.0
4,T001,S01,2013-01-10 00:40:00,267.0


## featuretools.EntitySet.normalize_entity

* Input: entityset
* Output: entityset with relationship (readings.turbine_id with turbines.turbine_id)
* Effect: establish relation between readings and turbines

In [14]:
step = 2
context = pipeline.fit(**context, output_=step, start_=step)

In [15]:
context.keys()

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])

In [16]:
context['entityset']

Entityset: entityset
  Entities:
    readings [Rows: 1329146, Columns: 5]
    turbines [Rows: 1, Columns: 1]
  Relationships:
    readings.turbine_id -> turbines.turbine_id

## featuretools.EntitySet.normalize_entity

* Input: entityset
* Output: entityset with relationship (readings.signal_id with signals.signal_id)
* Effect: establish relationship between readings and signals

In [17]:
step = 3
context = pipeline.fit(**context, output_=step, start_=step)

In [18]:
context.keys()

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])

In [19]:
context['entityset']

Entityset: entityset
  Entities:
    readings [Rows: 1329146, Columns: 5]
    turbines [Rows: 1, Columns: 1]
    signals [Rows: 26, Columns: 1]
  Relationships:
    readings.turbine_id -> turbines.turbine_id
    readings.signal_id -> signals.signal_id

## featuretools.dfs

* Input: entityset (unstacked, no turbine_id, no timestamp)
* Output: X (has additional features)
* Effect: build features for relational dataset using DFS

In [20]:
step = 4
context = pipeline.fit(**context, output_=step, start_=step)

In [21]:
context.keys()

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])

In [22]:
context['X'].head()

,COUNT(readings),MAX(readings.value),MEAN(readings.value),MIN(readings.value),MODE(readings.signal_id),NUM_UNIQUE(readings.signal_id),SKEW(readings.value),STD(readings.value),SUM(readings.value),MODE(readings.DAY(timestamp)),...,SKEW(readings.signals.NUM_UNIQUE(readings.WEEKDAY(timestamp))),SKEW(readings.signals.NUM_UNIQUE(readings.YEAR(timestamp))),STD(readings.signals.NUM_UNIQUE(readings.DAY(timestamp))),STD(readings.signals.NUM_UNIQUE(readings.MONTH(timestamp))),STD(readings.signals.NUM_UNIQUE(readings.WEEKDAY(timestamp))),STD(readings.signals.NUM_UNIQUE(readings.YEAR(timestamp))),SUM(readings.signals.NUM_UNIQUE(readings.DAY(timestamp))),SUM(readings.signals.NUM_UNIQUE(readings.MONTH(timestamp))),SUM(readings.signals.NUM_UNIQUE(readings.WEEKDAY(timestamp))),SUM(readings.signals.NUM_UNIQUE(readings.YEAR(timestamp)))
turbine_id,,,,,,,,,,,,,,,,,,,,,
T001,3744,3448719.0,917107.079193,0.0,S01,26,1.019214,1.456860e+06,3.433649e+09,11,...,0,0,0.0,0.0,0.0,0.0,7488,3744,7488,3744
T001,3744,3453777.0,919201.162179,0.0,S01,26,1.018761,1.459865e+06,3.441489e+09,12,...,0,0,0.0,0.0,0.0,0.0,7488,3744,7488,3744
T001,3744,3463880.0,922935.352244,2.7,S01,26,1.018192,1.465277e+06,3.455470e+09,13,...,0,0,0.0,0.0,0.0,0.0,7488,3744,7488,3744
T001,3744,3474703.0,928248.092869,-1.0,S01,26,1.017666,1.473337e+06,3.475361e+09,14,...,0,0,0.0,0.0,0.0,0.0,7488,3744,7488,3744
T001,3744,3485019.0,924186.531200,0.0,S01,26,1.032002,1.477958e+06,2.888083e+09,15,...,0,0,0.0,0.0,0.0,0.0,7488,3744,7488,3744


In [23]:
# features generated (the turbine_id is set as index).
len(context['X'].columns)

99

In [24]:
context['readings'].head()

,turbine_id,signal_id,timestamp,value
0,T001,S01,2013-01-10 00:00:00,323.0
1,T001,S01,2013-01-10 00:10:00,346.0
2,T001,S01,2013-01-10 00:20:00,407.0
3,T001,S01,2013-01-10 00:30:00,257.0
4,T001,S01,2013-01-10 00:40:00,267.0


## mlprimitives.custom.feature_extraction.CategoricalEncoder

* Input: X
* Output: X (label encoded)
* Effect: encodes categorical features using OneHotLabelEncoder

In [25]:
step = 5
context = pipeline.fit(**context, output_=step, start_=step)

In [26]:
context.keys()

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])

In [27]:
context['X'].head()

,COUNT(readings),MAX(readings.value),MEAN(readings.value),MIN(readings.value),NUM_UNIQUE(readings.signal_id),SKEW(readings.value),STD(readings.value),SUM(readings.value),MODE(readings.DAY(timestamp)),MODE(readings.MONTH(timestamp)),...,STD(readings.signals.NUM_UNIQUE(readings.DAY(timestamp))),STD(readings.signals.NUM_UNIQUE(readings.MONTH(timestamp))),STD(readings.signals.NUM_UNIQUE(readings.WEEKDAY(timestamp))),STD(readings.signals.NUM_UNIQUE(readings.YEAR(timestamp))),SUM(readings.signals.NUM_UNIQUE(readings.DAY(timestamp))),SUM(readings.signals.NUM_UNIQUE(readings.MONTH(timestamp))),SUM(readings.signals.NUM_UNIQUE(readings.WEEKDAY(timestamp))),SUM(readings.signals.NUM_UNIQUE(readings.YEAR(timestamp))),MODE(readings.signal_id)=S01,MODE(readings.signals.MODE(readings.turbine_id))=T001
turbine_id,,,,,,,,,,,,,,,,,,,,,
T001,3744,3448719.0,917107.079193,0.0,26,1.019214,1.456860e+06,3.433649e+09,11,1,...,0.0,0.0,0.0,0.0,7488,3744,7488,3744,1,1
T001,3744,3453777.0,919201.162179,0.0,26,1.018761,1.459865e+06,3.441489e+09,12,1,...,0.0,0.0,0.0,0.0,7488,3744,7488,3744,1,1
T001,3744,3463880.0,922935.352244,2.7,26,1.018192,1.465277e+06,3.455470e+09,13,1,...,0.0,0.0,0.0,0.0,7488,3744,7488,3744,1,1
T001,3744,3474703.0,928248.092869,-1.0,26,1.017666,1.473337e+06,3.475361e+09,14,1,...,0.0,0.0,0.0,0.0,7488,3744,7488,3744,1,1
T001,3744,3485019.0,924186.531200,0.0,26,1.032002,1.477958e+06,2.888083e+09,15,1,...,0.0,0.0,0.0,0.0,7488,3744,7488,3744,1,1


In [28]:
context['readings'].head()

,turbine_id,signal_id,timestamp,value
0,T001,S01,2013-01-10 00:00:00,323.0
1,T001,S01,2013-01-10 00:10:00,346.0
2,T001,S01,2013-01-10 00:20:00,407.0
3,T001,S01,2013-01-10 00:30:00,257.0
4,T001,S01,2013-01-10 00:40:00,267.0


## xgboost.XGBClassifier

* Input: X (label encoded and featurized)
* Output: None
* Effect: trained model

In [29]:
step = 6
context = pipeline.fit(**context, output_=step, start_=step)

In [30]:
context.keys()

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])